# Vocabulary and embedding

In [1]:
import pandas as pd
import numpy as np

In [2]:
from ast import literal_eval

Now that we have text pre-processed, we can create the vocabulary and embedding.

There are many different options, which embedding to use - for example doc2vec, glove, fastext, .. for the current first iteration of our project, we are going to go with Facebook's fastext. 

In [3]:
# load data
# df = pd.read_csv('../data/train/comments_processed.csv', index_col=0)
# df_test = pd.read_csv('../data/test/comments_processed.csv', index_col=0)

In [3]:
df = pd.read_csv('../data/train/comments_processed_v2.csv', index_col=0)
df_test = pd.read_csv('../data/test/comments_processed_v2.csv', index_col=0)

In [4]:
df.comment = df.comment.apply(literal_eval)
df_test.comment = df_test.comment.apply(literal_eval)

In [6]:
df.head(n=5)

,comment,sentiment
0,"[for, movi, that, get, respect, there, sure, a...",1
1,"[bizarr, horror, movi, fill, with, famou, face...",1
2,"[solid, unremark, film, matthau, einstein, won...",1
3,"[strang, feel, sit, alon, theater, occupi, par...",1
4,"[you, probabl, all, alreadi, know, thi, now, b...",1


We will create the vocabulary by assigning every word its number. 
However, we will sort the words according to their "priority", which is simply number of word occurences. 
This is only being done for the purpose of potentially limiting the size of vocabulary (e.g. to 15000 most frequently used words). We will see, if it makes sense later.

Also, another option is to use already built vocabulary along with already trained and prepared embeddings. 
This has an advantage of covering most of the words for english language. However, there might be slight disadvantage that the embedding is not domain specific - and people might communicate movie reviews a little bit differently than other texts. 

We can compare mulitple approaches later. 

In [5]:
# create vocabulary
words = {}
for idx, comment in df.comment.iteritems():
    for word in comment:
        if word in words.keys():
            words[word] += 1
        else:
            words[word] = 1

Order by occurences 

In [6]:
words = [(k, v) for k, v in words.items()]

In [7]:
words = sorted(words, key=lambda x: x[1], reverse=True)

In [10]:
words[:30]

[('the', 334779),
 ('and', 162257),
 ('thi', 75456),
 ('that', 73306),
 ('not', 63580),
 ('movi', 50595),
 ('film', 47239),
 ('for', 44022),
 ('with', 43951),
 ('but', 41765),
 ('have', 31024),
 ('are', 30243),
 ('you', 29637),
 ('one', 26912),
 ('all', 23387),
 ('like', 22185),
 ('who', 21099),
 ('they', 20965),
 ('from', 20439),
 ('there', 18591),
 ('her', 18401),
 ('just', 17638),
 ('about', 17322),
 ('out', 16561),
 ('what', 16082),
 ('some', 15686),
 ('time', 15542),
 ('good', 14924),
 ('can', 14569),
 ('make', 14546)]

In [11]:
words[-30:]

[('firsthalf', 1),
 ('drosselmei', 1),
 ('closedforweekend', 1),
 ('highsecur', 1),
 ('warveteran', 1),
 ('milla', 1),
 ('antithril', 1),
 ('descriptionwoman', 1),
 ('securityi', 1),
 ('realizationi', 1),
 ('quarterfin', 1),
 ('rakishli', 1),
 ('untempt', 1),
 ('crumley', 1),
 ('buic', 1),
 ('antiplot', 1),
 ('fictiondrama', 1),
 ('thereinaft', 1),
 ('lockup', 1),
 ('dresssuit', 1),
 ('overvot', 1),
 ('ontyp', 1),
 ('infantalis', 1),
 ('rou', 1),
 ('orientalist', 1),
 ('tooand', 1),
 ('repleat', 1),
 ('jowl', 1),
 ('camora', 1),
 ('capich', 1)]

In [12]:
len(words)

79771

As probably expected, we can see that for example occurence of word "movi" (stem from "movie") is almost the same as number of documents (even higher). So it goes for word film. I think these two words might be domain stopwords - frequently used and there with no added value to comments. 

What is interesting is that words "good" and "like" made it pretty high. However, they might have been used with not - so they alone (without context) are not helpful (this might be an example of why "not" should not be in the englighs stopwords for this problem. 

Another thing is that we surely have a lot of words, that occur once in all the comments. 
Our vocabulary is on the other hand quite large - almost 80 000. 
There comes the question, if it makes sense to leave every word in vocabulary - or if we want to use only words that occur for example at least 2-3 times (or more). 

Further analysis, like tf-idf computing, might help us understand the most meaningful words for our model, but for now, we will just skip words that occured a little times. 

In [13]:
words[12500]

('aubrey', 12)

In [14]:
words[15001]

('inter', 9)

In [15]:
words[25000]

('phedon', 3)

In [16]:
words[35000]

('serrador', 2)

Here we can also see that our vocab also consist of typos. 
After first most frequent 15000 words we can see that words occur no more than 9 times (from the whole set of 25000 movies). We can here try to limit our vocab to 15000 most ferquent words (with the risk od decreasing accuracy). 

We can also play with this parameter later - this is now done to simplyfy things a liitle and to speed up training as well. 

In [9]:
words = words[:15000]

In [10]:
vocab = {}
# assign words a number
for idx, word in enumerate(words):
    vocab[word[0]] = idx + 1

In [11]:
[(k, v) for k, v in vocab.items()][:20]  # not the most efective way :) 

[('the', 1),
 ('and', 2),
 ('thi', 3),
 ('that', 4),
 ('not', 5),
 ('movi', 6),
 ('film', 7),
 ('for', 8),
 ('with', 9),
 ('but', 10),
 ('have', 11),
 ('are', 12),
 ('you', 13),
 ('one', 14),
 ('all', 15),
 ('like', 16),
 ('who', 17),
 ('they', 18),
 ('from', 19),
 ('there', 20)]

Now that we have our vocabulary constructed, we can replace words with their ids.

In [11]:
df['comment_ids'] = df.comment.apply(lambda comm: list(map(lambda x: vocab.get(x, None), comm)))

Since we created the vocabulary ourselves and on train data set only, here can happen that test data set contains words that are new (not present in train data set). We will remove the words from comments as well. 

In [12]:
df_test['comment_ids'] = df_test.comment.apply(lambda comm: list(map(lambda x: vocab.get(x, None), comm)))

In [13]:
df['comment_ids'] = df.comment_ids.apply(lambda comm: list(filter(lambda x: x is not None, comm)))
df_test['comment_ids'] = df_test.comment_ids.apply(lambda comm: list(filter(lambda x: x is not None, comm)))

In [14]:
df_test.head(n=3)

,comment,sentiment,comment_ids
0,"[base, actual, stori, john, boorman, show, the...",1,"[404, 121, 40, 291, 9299, 54, 1, 844, 259, 866..."
1,"[thi, gem, film, four, product, the, anticip, ...",1,"[3, 1229, 7, 698, 288, 1, 2434, 441, 830, 596,..."
2,"[realli, like, thi, show, drama, romanc, and, ...",1,"[45, 16, 3, 54, 447, 797, 2, 171, 15, 929, 64,..."


In [15]:
df.head(n=10)

,comment,sentiment,comment_ids
0,"[for, movi, that, get, respect, there, sure, a...",1,"[8, 6, 4, 33, 694, 20, 207, 12, 126, 831, 1649..."
1,"[bizarr, horror, movi, fill, with, famou, face...",1,"[1049, 175, 6, 702, 9, 781, 300, 10, 2269, 684..."
2,"[solid, unremark, film, matthau, einstein, won...",1,"[1081, 7097, 7, 2611, 4724, 166, 453, 118, 2, ..."
3,"[strang, feel, sit, alon, theater, occupi, par...",1,"[551, 117, 500, 580, 581, 3875, 676, 2, 47, 13..."
4,"[you, probabl, all, alreadi, know, thi, now, b...",1,"[13, 223, 15, 459, 84, 3, 130, 10, 1089, 243, ..."
5,"[saw, the, movi, with, two, grown, children, a...",1,"[199, 1, 6, 9, 89, 2061, 430, 254, 5, 1005, 62..."
6,"[your, use, the, imdb, youv, given, some, heft...",1,"[82, 120, 1, 887, 827, 357, 26, 10388, 1365, 2..."
7,"[thi, good, film, with, power, messag, love, a...",1,"[3, 28, 7, 9, 347, 635, 70, 2, 2626, 70, 1, 15..."
8,"[made, after, quartet, trio, continu, the, qua...",1,"[77, 83, 8771, 3224, 531, 1, 441, 1, 880, 7, 2..."
9,"[for, matur, man, admit, that, shed, tear, ove...",1,"[8, 1885, 107, 805, 4, 2566, 1157, 109, 3, 7, ..."


This is the part where we will create and train our Fastext (or another) embedding, or just use already trained one (and comapre results then). 
Since this is only the first iteration on our project, we are ending here and letting our Neural network create and train the embedding - even if it does not capture the word relations. 
We can then compare, how using advanced embeddings helps us to achieve better results. 

##### Project v2 - training embedding

In [12]:
import fasttext

In [18]:
# dump text into text file
df['comment_text'] = df.comment.apply(lambda x: ' '.join(x))

In [19]:
comment_text = df.comment_text.values.tolist()

In [21]:
with open('text_input.in', 'w') as write_file:
    for text in comment_text:
        write_file.write(f'{text}\n')

Fasttext embedding offers two types of representation - skipgram and cbow. We chose to train skipgram, since those models should be better at predicting context - and that's what we would like to try to get into our model as well. 

In [22]:
# train model
emb_model = fasttext.train_unsupervised('text_input.in', model='skipgram')

In [23]:
emb_model.save_model("fasttext_model.bin")

In [13]:
emb_model = fasttext.load_model("fasttext_model.bin")

In [19]:
emb_model.get_word_vector('and').shape

(100,)

In [21]:
embedding = np.zeros((len(vocab) + 1, 100))

In [23]:
for k, v in vocab.items():
    embedding[v] = emb_model.get_word_vector(k)

In [25]:
embedding[1:3]

array([[ 0.09602074, -0.35753947, -0.03201434, -0.33081603, -0.0247028 ,
         0.08232363, -0.0059217 ,  0.02468209,  0.08795767, -0.18481494,
        -0.03276435,  0.28888649, -0.11125953, -0.1052271 ,  0.13869125,
         0.11167844, -0.15603273, -0.08394193, -0.01224886,  0.17099385,
        -0.28143308, -0.17877622,  0.12219147,  0.00517188, -0.05449152,
        -0.02559651, -0.03564524,  0.09680374,  0.05482788, -0.11976379,
        -0.158576  , -0.01492924,  0.00803954,  0.10995729,  0.10254253,
        -0.31437603, -0.01514904, -0.12867044, -0.09065547, -0.11344571,
         0.01505242, -0.19488128, -0.06491195,  0.25242415,  0.24487579,
        -0.15954022, -0.12156445, -0.10933612, -0.2566855 , -0.07177238,
         0.21224183,  0.11129531, -0.30246639, -0.0419183 ,  0.08962663,
        -0.0897903 ,  0.28208923,  0.20725051, -0.14286117, -0.35964763,
        -0.07835703, -0.05237986,  0.01240221,  0.16900781,  0.00564027,
        -0.12043378, -0.08101835, -0.24174491,  0.0

We can store this embedding and use it within model later.

In [26]:
np.save('word_embeddings.npy', embedding)

We also still need to make sure that our documents have are the same length. Let's do a quick analysis of lengths first:

In [16]:
# compute comment lengths
df['words_n'] = df.comment_ids.apply(len)

In [17]:
df.words_n.describe()

count    25000.000000
mean       112.184480
std         84.741327
min          4.000000
25%         60.000000
50%         83.000000
75%        137.000000
max       1320.000000
Name: words_n, dtype: float64

In [133]:
df.words_n.quantile(0.95)

291.0

In [134]:
df.words_n.quantile(0.90)

223.0

In [18]:
df.words_n.quantile(0.85)

183.0

Only a smaller part of our data set (slightly more than 10\%) contains more than 200 words. 
Since we think that 100 words could be enough to detect sentiment, we will set that as maximum length of our comment and cut the words in comments after. However, this is a parameter to play with later. 

In [19]:
# set 100 as maximum comment length
df['x'] = df.comment_ids.apply(lambda x: x[:100])
df_test['x'] = df_test.comment_ids.apply(lambda x: x[:100])

In [20]:
# pad shorter comments with 0
df['x'] = df.x.apply(lambda x: np.pad(x, (0, 100 - len(x)), mode='constant'))
df_test['x'] = df_test.x.apply(lambda x: np.pad(x, (0, 100 - len(x)), mode='constant'))

And we are ready to go for now.

In [21]:
# length of vocabulary for further processing
len(vocab)

15000

In [22]:
# persist - to pickle now
df.to_pickle('../data/train/comments_embed.pkl')
df_test.to_pickle('../data/test/comments_embed.pkl')

In [23]:
del df
del df_test